<a href="https://colab.research.google.com/github/davidfague/parallel-neuron/blob/main/cs4001_mpi_matrix_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/cyneuro/CI-BioEng-Class/blob/main/cs4001_mpi.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Introduction to MPI

## Homework:

1. Review the MPI parallelization of the matrix multiplication problem in [this tutorial](https://afzalbadshah.medium.com/matrix-multiplication-on-multiple-processors-mpi4py-dce0cb4a6d53).
2. Using the code above, write a function which can multiply 2 random $N\times N$ matrices.
3. Select 3-4 values of $N$ (e.g., $N =$ 10, 100, 500, 1000) and record multiplication time in the following scenarios:
    - Serial multiplication on a PC.
    - Parallel multiplication on a PC.
    - Parallel multiplication in Colab.
    - Parallel multiplication on FABRIC / ACCESS / CloudLab / Hellbender.

Plot the simulation time dynamics of these scenarios on the same graph.

In [1]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458236 sha256=3102303d9ebb2ca169e980d1e807fa65068bfaa1b743cb711cd652a82980b476
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [2]:
from mpi4py import MPI
import numpy as np
import time

In [3]:
# Function to perform matrix multiplication
def matrix_multiply(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i][j] += A[i][k] * B[k][j]
    return C

In [4]:
# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [8]:
location = 'Google Colab'
Ns = [10,100,500,1000]
times = []
parallels = []
locations = []
parallel_options = [False, True]
for parallel in parallel_options:
  for N in Ns:
    start_time = time.time()
    if parallel:
      # Master process
      if rank == 0:
          # Generate matrices A and B
          A = np.random.rand(N, N)
          B = np.random.rand(N, N)

          # Split matrices for distribution
          chunk_size = A.shape[0] // size
          A_chunks = [A[i:i+chunk_size] for i in range(0, A.shape[0], chunk_size)]

          # Send parts of A and B to worker processes
          for i in range(1, size):
              comm.send(A_chunks[i-1], dest=i, tag=1)
              comm.send(B, dest=i, tag=2)

          # Calculate its own part of multiplication
          C_partial = matrix_multiply(A_chunks[0], B)

          # Collect results from worker processes
          for i in range(1, size):
              C_partial += comm.recv(source=i, tag=3)

          # Print the resulting matrix
          # print("Resulting matrix C:")
          # print(C_partial)
      # Worker processes
      else:
          # Receive matrix chunks from master
          A_chunk = comm.recv(source=0, tag=1)
          B = comm.recv(source=0, tag=2)

          # Perform multiplication
          C_partial = matrix_multiply(A_chunk, B)

          # Send back the result to master
          comm.send(C_partial, dest=0, tag=3)
    else:
      if rank == 0:
        # Generate matrices A and B
        A = np.random.rand(N, N)
        B = np.random.rand(N, N)
        C = matrix_multiply(A, B)
    end_time = time.time()
    times.append(end_time - start_time)
    parallels.append(parallel)
    locations.append(location)
    print(f"Execution time: {end_time - start_time} seconds for N = {N} {'parallel' if parallel else 'serial'}")

# end_time = time.time()
# times.append(end_time - start_time)
# print(f"Execution time: {end_time - start_time} seconds for N = {N}")

Execution time: 0.0025856494903564453 seconds for N = 10
Execution time: 0.961047887802124 seconds for N = 100
Execution time: 141.30803632736206 seconds for N = 500
Execution time: 1112.2218866348267 seconds for N = 1000
Execution time: 0.0017328262329101562 seconds for N = 10
Execution time: 0.9827353954315186 seconds for N = 100
Execution time: 139.39754366874695 seconds for N = 500
Execution time: 1126.6178255081177 seconds for N = 1000


In [23]:
# dataframe of times and Ns
import pandas as pd
df = pd.DataFrame({'NxN': Ns * len(parallel_options), 'Time': times, 'Parallelization':parallels, 'Location':locations, 'mpi_size':[size for time in times]})
df.to_csv('Results_Google_Colab.csv')

In [22]:
size

1